In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import pymysql
import pandas as pd

conn = pymysql.Connection(
    host = 'mml.cu4cw1rqzfei.ap-northeast-2.rds.amazonaws.com',
    user = 'admin',
    password = 'pizza715',
    database = 'mml',
    charset = 'utf8')

mml_music_info = 'SELECT * FROM mml_music_info'
mml_music_info_df = pd.read_sql(mml_music_info, conn)

mml_artist_gen = 'SELECT * FROM mml_artist_gen'
mml_artist_gen_df = pd.read_sql(mml_artist_gen, conn)

mml_user_like_artist = 'SELECT * FROM mml_user_like_artist'
mml_user_like_artist_df = pd.read_sql(mml_user_like_artist, conn)

C:\Users\AIschool\AppData\Local\Temp\ipykernel_35068\726121239.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mml_music_info_df = pd.read_sql(mml_music_info, conn)
C:\Users\AIschool\AppData\Local\Temp\ipykernel_35068\726121239.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mml_artist_gen_df = pd.read_sql(mml_artist_gen, conn)
C:\Users\AIschool\AppData\Local\Temp\ipykernel_35068\726121239.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mml_user_like_artist_df = pd.read_sql(mml_user_like_artist, conn)


In [3]:
mml_user_like_artist_df.head()
mml_user_like_artist_df = mml_user_like_artist_df.rename(columns={'artist_id':'artist'})
mml_user_like_artist_df.columns

Index(['id', 'gen', 'age_group', 'artist', 'user_id'], dtype='object')

In [4]:
# 데이터 전처리
# 사용자가 좋아하는 아티스트 데이터와 아티스트 장르 데이터를 병합하여 좋아하는 아티스트의 장르를 구합니다.
merged_data = pd.merge(mml_user_like_artist_df, mml_artist_gen_df, on='artist', how='left')

merged_data.head()

,id,gen,age_group,artist,user_id,priority,genre
0,52716,F,10대,BTS,00daK56D,1.0,POP / 랩/힙합
1,52716,F,10대,BTS,00daK56D,1.0,POP / 랩/힙합
2,52717,F,10대,IVE (아이브),00daK56D,NaN,NaN
3,52718,F,10대,TWICE (트와이스),00daK56D,1.0,가요 / 댄스
4,52719,F,10대,aespa,00daK56D,NaN,NaN


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 전처리
# 사용자가 좋아하는 아티스트 데이터와 아티스트 장르 데이터를 병합하여 좋아하는 아티스트의 장르를 구합니다.
merged_data = pd.merge(mml_user_like_artist_df, mml_artist_gen_df, on='artist', how='left')

# 사용자별로 데이터를 그룹화하고 좋아하는 모든 장르, 성별, 연령 그룹을 연결합니다.
user_genre_df = merged_data.groupby('user_id').agg({
    'genre': lambda x: ' '.join(x.dropna()),
    'gen': 'first',    # 가정: 모든 행에 대해 동일한 값이 존재한다고 가정
    'age_group': 'first'  # 가정: 모든 행에 대해 동일한 값이 존재한다고 가정
}).reset_index()


# ITF-IDF 벡터 구현
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(user_genre_df['genre'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 사용자 ID를 기반으로 노래를 추천하는 기능 (성별 및 연령대 필터링 적용)
def recommend_songs(user_id, num_recommendations=3):
    # 사용자 id와 동일한 성별 및 연령대를 가진 사용자들만 필터링
    target_user_data = user_genre_df[user_genre_df['user_id'] == user_id]
    if target_user_data.empty:
        return "사용자 ID를 찾을 수 없습니다."

    target_gen = target_user_data['gen'].iloc[0]
    target_age_group = target_user_data['age_group'].iloc[0]

    filtered_users = user_genre_df[(user_genre_df['gen'] == target_gen) &
                                   (user_genre_df['age_group'] == target_age_group)]

    # 필터링된 사용자들의 인덱스 추출
    filtered_user_indices = filtered_users.index.tolist()

    # 해당 사용자와 필터링된 사용자들의 유사성 점수를 가져옵니다.
    idx = user_genre_df.index[user_genre_df['user_id'] == user_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사성 점수 중에서 필터링된 사용자들만을 대상으로 정렬
    sim_scores = [sim_score for sim_score in sim_scores if sim_score[0] in filtered_user_indices]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 사용자들의 점수
    sim_scores = sim_scores[1:num_recommendations+1]

    # 사용자 인덱스
    user_indices = [i[0] for i in sim_scores]

    # 가장 유사한 사용자들 반환
    return user_genre_df['user_id'].iloc[user_indices]

# 테스트
test_user_id = 'Hvuta2FS'  # 예시 사용자
recommended_users = recommend_songs(test_user_id)
print(recommended_users)

5728    mRkDuPgp
3862    WdrhMwPQ
5623    lWimKImA
Name: user_id, dtype: object


In [6]:
# 추천 함수를 수정하여 이미 선호하는 아티스트를 제외하고 추천
def recommend_new_artists(user_id, num_recommendations=10):

    # 원래 사용자의 아티스트 선호도 가져오기
    user_artists = mml_user_like_artist_df[mml_user_like_artist_df['user_id'] == user_id]['artist'].tolist()

    # 사용자 id에 기반한 추천 받기
    recommended_user_ids = recommend_songs(user_id, num_recommendations).tolist()

    # 추천된 사용자들이 선호하는 아티스트 찾기
    recommended_artists = mml_user_like_artist_df[mml_user_like_artist_df['user_id'].isin(recommended_user_ids)]['artist']

    # 원래 사용자가 선호하는 아티스트 제외
    new_recommended_artists = recommended_artists[~recommended_artists.isin(user_artists)].unique()

    return new_recommended_artists

# 이제 추천 함수를 다시 실행하여 테스트 사용자와 유사한 사용자를 찾을 수 있습니다.
recommended_user_ids = recommend_songs(test_user_id).tolist()

# 유사한 사용자들이 선호하는 아티스트 찾기
preferred_artists = mml_user_like_artist_df[mml_user_like_artist_df['user_id'].isin(recommended_user_ids)]['artist'].unique()

preferred_artists

array(['015B', 'P.O (피오)', '이효리', '김태우', '바로', '윤하 (YOUNHA)',
       '(G)I-dle ((여자)아이들)', 'AKMU (악동뮤지션)', 'BTS', '잔나비 최정훈',
       '에일리(AILEE)', 'PSY (싸이)', 'BLACKPINK', '여자친구 (GFRIEND)', '청하',
       '태연 (TAEYEON)', '마마무 (Mamamoo)', '폴킴', '이무진', '정동원', 'ITZY (있지)',
       'TWICE (트와이스)', '벤', '오마이걸 (OH MY GIRL)', '세븐틴', '임영웅', '이영지',
       '정국', 'SHINee (샤이니)', '브레이브걸스', 'Apink (에이핑크)', 'Ariana Grande',
       '코요태', '정승환', '이수현', '규현 (KYUHYUN)', '장윤정', 'Red Velvet (레드벨벳)',
       '전소미', 'Jessi (제시)', 'MINO (송민호)', '이달의 소녀', '카라', '홍진영', '영탁',
       '스탠딩 에그', '정용화 (CNBLUE)', '현아', '에이프릴 (APRIL)', '태양', '경서 (경서예지)',
       '잔나비', '던 (DAWN)', '소진 (Sojin)', 'Vivivavy (비비베이비)', 'PRISTIN V ',
       '이대휘 (AB6IX)', '박우진 (AB6IX)', '이달의 소녀 yyxy', '정예원 (YEWON)',
       'YELO (옐로)', '유진', '김윤아', '채리나', '옹성우', '설리 (SULLI)', 'ASH ISLAND',
       '하니 (EXID)', '윤채경 (에이프릴)', '유노윤호 (U-KNOW)', '민서', '솔라 (마마무)',
       'Bin boy', '크리스탈 (KRYSTAL)', '윤보미', '융 (Jung)', 'WABLE (와블)',
       '인석 (Inseok)

In [11]:
def get_all_songs_for_artists(artist_list):
    songs_dict = {}
    for artist in artist_list:
        # 해당 아티스트의 모든 노래만 필터링
        artist_songs = mml_music_info_df[mml_music_info_df['artist'] == artist]
        # 노래가 존재하는 경우에만 딕셔너리에 추가
        if not artist_songs.empty:
            songs_dict[artist] = artist_songs['title'].tolist()
    return songs_dict

# 모든 추천된 아티스트별로 모든 노래를 가져옵니다.
artist_songs_dict = get_all_songs_for_artists(preferred_artists)

# 랜덤으로 20곡 선택하기 전에 각 곡에 해당하는 아티스트 정보도 포함시키기
all_songs_with_artists = []
for artist, songs in artist_songs_dict.items():
    for song in songs:
        all_songs_with_artists.append((artist, song))

# 랜덤으로 20곡 선택 (아티스트 정보 포함)
selected_songs_with_artists = random.sample(all_songs_with_artists, 20)

# 선택된 노래와 아티스트를 데이터프레임으로 변환
df_selected_songs_with_artists = pd.DataFrame(selected_songs_with_artists, columns=['artist', 'title'])
df_selected_songs_with_artists.head(20)  # 첫 5곡과 아티스트 출력하여 확인

,artist,title
0,정국,3d (feat. jack harlow) - clean ver.
1,영탁,니가 왜 거기서 나와
2,러블리즈,걱정 인형
3,성시경,광화문 연가
4,드렁큰 타이거,skit #5
5,걸스데이,i don't mind
6,민서,the first love
7,나윤권,10분 거리인데
8,임영웅,despacito
9,이효리,묻지 않을게요


In [12]:
# Normalize the 'Title' and 'Artist' columns in both dataframes for case-insensitive comparison
mml_music_info_df['title'] = mml_music_info_df['title'].str.lower()
mml_music_info_df['artist'] = mml_music_info_df['artist'].str.lower()
df_selected_songs_with_artists['title'] = df_selected_songs_with_artists['title'].str.lower()
df_selected_songs_with_artists['artist'] = df_selected_songs_with_artists['artist'].str.lower()

# Merge the dataframes on 'Title' and 'Artist' to find matching songs
song2vec_final = pd.merge(
    mml_music_info_df, df_selected_songs_with_artists,
    on=['title', 'artist'],
    how='inner'
)

# Since the user wants to add lyrics to the user music log based on the title and artist match,
# we will merge the 'user_music_log_with_genres' with 'new_all_music_data' on 'Title' and 'Artist'
# to add the 'Lyrics' column to the user music log dataframe.

# We will use the 'matched_songs_df' which already has the matched songs to select the needed columns
# We will create a new dataframe with the 'Lyrics' column included

# Selecting only the necessary columns to include in the final merged dataframe
song2vec_final = song2vec_final[['title', 'artist', 'genre', 'lyrics', 'album_image_url']]

song2vec_final.head(20)

,title,artist,genre,lyrics,album_image_url
0,10분 거리인데,나윤권,가요 / 발라드,나 잠깐 갈까 너 있는 곳으로'\n또 다시 안 받을 전화를 걸곤 해\n울컥대는 버스...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
1,3d (feat. jack harlow) - clean ver.,정국,가요 / 댄스,"1, 2, 3D\n \nI can’t touch you through the pho...",https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
2,breeze,어반자카파,가요 / R&B/소울,찬 바람이 불어와 \n날 감싸고 또 떠나 그대처럼\n딴 사랑이 또 와도 \n그때 우...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
3,despacito,임영웅,가요 / 트로트,Ay\nFonsi\nDY\nOhhh\nOh no oh no\nHey yeah\n\n...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
4,golden slumbers,이하이,OST / 전체,Once there was a way\nTo get back homeward\nOn...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
5,i don't mind,걸스데이,가요 / 발라드,별이 빛나요 그댈 위한 \nTonight Tonight Tonight\n날 깨워준 ...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
6,no sympathy,손담비,가요 / 댄스,아직 거짓말처럼 아무 일도 없듯이 다시 넌 내게 돌아왔지\n하지만 나도 그렇게 아무...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/07...
7,skit #5,드렁큰 타이거,가요 / 랩/힙합,트름 꺼으으으윽~~)rnrn(신호음 뚜뚜우~~ )rnrn여보세요~ rnrn네~rnr...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/01...
8,the first love,민서,OST / 전체,너의 눈빛이 보여\n너의 마음이 들려\n아무런 말 하지 않아도\n다시 또 멀어지고\...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
9,걱정 인형,러블리즈,가요 / 댄스,숨소리만 들어도 난 느낄 수 있어\n네 어깨의 작은 뒤척임까지도\n밤새 얼룩진 베개...,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
